In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
class DBN_info():
    def __init__(self):
        self.view = 0
        self.click = 0
        self.satis = 0
def http_cut1(ex_str):
    ex_str = ex_str.replace('http://',"")
    ex_str = ex_str.replace('https://',"")
    ex_str = ex_str.replace('www.',"")
    if len(ex_str)>0:
        if ex_str[-1]=='/':
            ex_str=ex_str[:-1]
    return ex_str
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]

In [2]:
with open('query_dict/id_querry_clean.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [5]:
vals = all_groups['DocumentId'].values

In [6]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)

In [7]:
with open("url_dbn.pickle", 'rb') as file:
    url_dbn = pickle.load(file)
with open("host_dbn.pickle", 'rb') as file:
    host_dbn = pickle.load(file)

In [8]:
dbn_feats = dict()
dbn_feats['url_dbn'] = []
dbn_feats['url_dbn_a'] = []
dbn_feats['url_dbn_s'] = []
dbn_feats['url_dbn_v'] = []
dbn_feats['url_dbn_cl'] = []
dbn_feats['url_dbn_sat'] = []
dbn_feats['host_dbn'] = []
dbn_feats['host_dbn_a'] = []
dbn_feats['host_dbn_s'] = []
dbn_feats['host_dbn_v'] = []
dbn_feats['host_dbn_cl'] = []
dbn_feats['host_dbn_sat'] = []

In [9]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    a = (url_dbn[key1].click+0.1)/(url_dbn[key1].view+0.2)
    s = (url_dbn[key1].satis+0.1)/(url_dbn[key1].click+0.2)
    dbn_feats['url_dbn'].append(a*s)
    dbn_feats['url_dbn_a'].append(a)
    dbn_feats['url_dbn_s'].append(s)
    dbn_feats['url_dbn_v'].append(url_dbn[key1].view)
    dbn_feats['url_dbn_cl'].append(url_dbn[key1].click)
    dbn_feats['url_dbn_sat'].append(url_dbn[key1].satis)

100%|██████████| 606050/606050 [00:09<00:00, 63802.90it/s]


In [10]:
for key in tqdm(vals):
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    a = (host_dbn[key1].click+0.1)/(host_dbn[key1].view+0.2)
    s = (host_dbn[key1].satis+0.1)/(host_dbn[key1].click+0.2)
    dbn_feats['host_dbn'].append(a*s)
    dbn_feats['host_dbn_a'].append(a)
    dbn_feats['host_dbn_s'].append(s)
    dbn_feats['host_dbn_v'].append(host_dbn[key1].view)
    dbn_feats['host_dbn_cl'].append(host_dbn[key1].click)
    dbn_feats['host_dbn_sat'].append(host_dbn[key1].satis)

100%|██████████| 606050/606050 [00:07<00:00, 83856.58it/s] 


In [11]:
with open("dbn_feats_advanced.pickle", 'wb') as file:
    pickle.dump(dbn_feats, file)

In [12]:
2+7

9